In [1]:
from flask import Flask, request
import boto3
import paramiko
import os

app = Flask(__name__)

# Define the EC2 instance details
instance_id = 'i-0e40262d55bb2c481'  #instance id
key_pair_file = 'la.pem'   #pem key

session = boto3.Session(
    aws_access_key_id='your_access_key',
    aws_secret_access_key='your_secret_key',
    region_name='ap-south-1'
)

# Connect to the EC2 instance
ec2 = session.resource('ec2')
instance = ec2.Instance(instance_id)

# Get the public IP address of the instance
public_ip = instance.public_ip_address

# Connect to the instance over SSH using paramiko
key = paramiko.RSAKey.from_private_key_file(key_pair_file)
client = paramiko.SSHClient()
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
client.connect(hostname=public_ip, username='ec2-user', pkey=key)

@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        command = request.form.get('command')
        stdin, stdout, stderr = client.exec_command(command)
        output = stdout.read().decode() + stderr.read().decode()
        return output
    else:
        return '''
        <form method="post">
            Command: <input type="text" name="command">
            <input type="submit" value="Submit">
        </form>
        '''

if __name__ == '__main__':
    app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [09/Mar/2023 04:36:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2023 04:36:06] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [09/Mar/2023 04:36:10] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2023 04:36:19] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2023 04:36:24] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2023 04:36:32] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2023 04:37:47] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2023 04:37:49] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2023 04:37:53] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2023 04:37:55] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2023 04:38:01] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2023 04:38:06] "GET / HTTP/1.1" 200 -


In [ ]:
#!/bin/bash

# Get the third most CPU consuming process
cpu_info=$(ps aux --sort=-%cpu | awk '{if(NR==4)  print $11,$3,$4,$6,$2}' )

# Get the third most memory consuming process
mem_info=$(ps aux --sort=-%mem | awk '{if(NR==4)   print $11,$3,$4,$6,$2}')

# Output the information to a file
echo "Process Name % CPU used % Memory used PORT PID" > output.txt
echo $cpu_info >> output.txt
echo $mem_info >> output.txt
